### Convert the tlp file output by GrouseFlocks to MyJSON 
* Observing that all of the tlp files are at most two level, this script only deals with two level
* We treat each open metanode as a "true metanode" that has underlying leaf nodes, but the closed metanode does not have underlying leaf nodes.  They are both in the data structure "metanodes"

In [94]:
from tulip import tlp
from shapely.geometry import *
from tlp2myjson import chop_segment
import os
import math
import json
SQUARE_ROOT_2 = math.sqrt(2)

In [95]:
DATA_DIR = '../../../data/real-world-compiled/partition/'
FILENAME = 'partition-bcsstk33-grouseflocks-open-8'
grousetlp_filepath = os.path.join(DATA_DIR, FILENAME + '.tlp')
myjson_filepath = os.path.join(DATA_DIR, FILENAME + '.json')

In [96]:
graph = tlp.loadGraph(grousetlp_filepath)
hierarchy_subgraphs = graph.getSubGraph('1__HierarchySubGraphs')
hierarchy_unnamed = hierarchy_subgraphs.getSubGraph('unnamed')
final_layout = graph.getSubGraph('1__Final Layout Display')
assert(hierarchy_subgraphs is not None)
assert(hierarchy_unnamed)
assert(final_layout is not None)

In [97]:
metanodes = {}
leaf_nodes = []
edges = []
root = graph.getId()
# a mapping from lable "Contracted xx" in "Final Layout Display" to metanode id
label2metanode_id = {}

In [98]:
view_label = final_layout.getStringProperty('viewLabel')
view_disp2hier = final_layout.getIntegerProperty('view Display Node to Hier')
view_hier2disp = final_layout.getIntegerProperty('view Hier Node to Display')
view_layout = final_layout.getLayoutProperty('viewLayout')
view_size = final_layout.getSizeProperty('viewSize')

for n in final_layout.getNodes():
    center_coor = view_layout[n]
    width = view_size[n][0]
    if view_hier2disp[n] == -1:
        metanode_id = view_disp2hier[n]
        label2metanode_id[view_label[n]] = metanode_id
        metanodes[metanode_id] = {
            'id': metanode_id,   
            'parent_metanode': root,
            'desc_metanodes': {},
            'level': 0,
            'leaf_nodes': {},
            'diameter': width,
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(width / 2.0, cap_style=CAP_STYLE.round)
        }
print(label2metanode_id)

{'Contracted 0': 8740, 'Contracted 1': 8741, 'Contracted 2': 8742, 'Contracted 3': 8743, 'Contracted 4': 8744, 'Contracted 5': 8745, 'Contracted 6': 8746, 'Contracted 7': 8747, 'Contracted 8': 8748, 'Contracted 9': 8749, 'Contracted 10': 8750, 'Contracted 11': 8751, 'Contracted 12': 8752, 'Contracted 13': 8753, 'Contracted 14': 8754, 'Contracted 15': 8755, 'Contracted 16': 8756, 'Contracted 17': 8757, 'Contracted 18': 8758, 'Contracted 19': 8759, 'Contracted 20': 8760, 'Contracted 21': 8761, 'Contracted 22': 8762, 'Contracted 23': 8763, 'Contracted 24': 8764, 'Contracted 25': 8765, 'Contracted 26': 8766, 'Contracted 27': 8767, 'Contracted 28': 8768, 'Contracted 29': 8769, 'Contracted 30': 8770, 'Contracted 31': 8771, 'Contracted 32': 8772, 'Contracted 33': 8773, 'Contracted 34': 8774, 'Contracted 35': 8775, 'Contracted 36': 8776, 'Contracted 37': 8777, 'Contracted 38': 8778, 'Contracted 39': 8779, 'Contracted 40': 8780, 'Contracted 41': 8781, 'Contracted 43': 8783, 'Contracted 44': 878

In [99]:
# Find the coodinates and sizes for all metanodes (both open and closed)
sub_view_layout = hierarchy_unnamed.getLayoutProperty('viewLayout')
sub_view_size = hierarchy_unnamed.getSizeProperty('viewSize')
view_hier_attr = hierarchy_unnamed.getIntegerProperty('view Hier Node to Display')
view_meta_graph = graph.getGraphProperty('viewMetaGraph')
parents = {}

for n in hierarchy_unnamed.getNodes():
    center_coor = sub_view_layout[n]
    # Use the diagonal of a bounding square as the diameter of bounding circle 
    # (because it shows as rectangle instead of circle)
    diameter = sub_view_size[n][0] * SQUARE_ROOT_2
    
    if view_hier_attr[n] == -1:
        # indicate this is an open metanode
        metanodes[n.id] = {
            'id': n.id,   
            'parent_metanode': root,
            'desc_metanodes': {},
            'level': 0,
            'leaf_nodes': {},
            'diameter': diameter,
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(diameter / 2.0, cap_style=CAP_STYLE.round)
        }
        print('open metanode: ', n.id, view_meta_graph[n])
        for leaf in view_meta_graph[n].getNodes():
            metanodes[n.id]['leaf_nodes'][leaf.id] = True
            parents[leaf.id] = n.id
        
print(len(metanodes))
# print(metanodes.keys())

open metanode:  8782 <graph "Contracted 42" (id 252) >
open metanode:  8790 <graph "Contracted 50" (id 260) >
open metanode:  8809 <graph "Contracted 69" (id 279) >
open metanode:  8876 <graph "Contracted 136" (id 346) >
open metanode:  8877 <graph "Contracted 137" (id 347) >
open metanode:  8883 <graph "Contracted 143" (id 353) >
open metanode:  8890 <graph "Contracted 150" (id 360) >
open metanode:  8936 <graph "Contracted 196" (id 406) >
200


In [100]:
# Find out all leaf nodes
view_hier_attr = final_layout.getIntegerProperty('view Hier Node to Display')
sub_view_layout = final_layout.getLayoutProperty('viewLayout')
sub_view_size = final_layout.getSizeProperty('viewSize')
leaf_mapping = {}

for n in final_layout.getNodes():
    center_coor = sub_view_layout[n]
    width = sub_view_size[n][0]
    # print(n.id, center_coor, diameter)
    if view_hier_attr[n] != -1:
        # leaf nodes
        print('leaf: ', n.id)
        # Leaf node is only possible when a metanode is open
        assert(n.id in parents)      
        new_node = {
            'id': n.id,
            'parent_metanode': parents[n.id], 
            'geometry': Point(center_coor.x(), center_coor.y()).buffer(width / 2.0, cap_style=CAP_STYLE.square),
            'diameter': width * SQUARE_ROOT_2,
        }     
        leaf_mapping[n.id] = new_node
        leaf_nodes.append(new_node)

print(len(leaf_nodes))

leaf:  1538
leaf:  1539
leaf:  8463
leaf:  8468
leaf:  8469
leaf:  8472
leaf:  8476
leaf:  8477
leaf:  8479
leaf:  8480
leaf:  8481
leaf:  8482
leaf:  8483
leaf:  8486
leaf:  8487
leaf:  8488
leaf:  8490
leaf:  8491
leaf:  1572
leaf:  8495
leaf:  8499
leaf:  8500
leaf:  1580
leaf:  8502
leaf:  8503
leaf:  814
leaf:  815
leaf:  8507
leaf:  817
leaf:  8511
leaf:  8512
leaf:  824
leaf:  825
leaf:  826
leaf:  8517
leaf:  827
leaf:  8519
leaf:  829
leaf:  8521
leaf:  8522
leaf:  833
leaf:  5456
leaf:  5457
leaf:  847
leaf:  848
leaf:  850
leaf:  864
leaf:  8559
leaf:  5486
leaf:  5487
leaf:  5488
leaf:  5489
leaf:  5491
leaf:  882
leaf:  883
leaf:  5497
leaf:  886
leaf:  887
leaf:  890
leaf:  8598
leaf:  3984
leaf:  3985
leaf:  5525
leaf:  8603
leaf:  3989
leaf:  5528
leaf:  3990
leaf:  5529
leaf:  5530
leaf:  3994
leaf:  3995
leaf:  8634
leaf:  8635
leaf:  5565
leaf:  5566
leaf:  5567
leaf:  5568
leaf:  8645
leaf:  5569
leaf:  4032
leaf:  8647
leaf:  4034
leaf:  8649
leaf:  5573
leaf:  403

In [101]:
view_label = final_layout.getStringProperty('viewLabel')
view_layout = final_layout.getLayoutProperty('viewLayout')

for e in final_layout.getEdges():
    src, tgt = graph.ends(e)
    if src.id in leaf_mapping:
        src_id = src.id
        src_geo = leaf_mapping[src.id]['geometry']
    else:
        src_id = label2metanode_id[view_label[src]]
        src_geo = metanodes[src_id]['geometry']
        
    if tgt.id in leaf_mapping:
        tgt_id = tgt.id
        tgt_geo = leaf_mapping[tgt.id]['geometry']
    else:
        tgt_id = label2metanode_id[view_label[tgt]]
        tgt_geo = metanodes[tgt_id]['geometry']
        
    edge_id = '{}-{}'.format(src_id, tgt_id) 
    edges.append({
        'id': edge_id,
        'ends': (src_id, tgt_id),
        'geometry': chop_segment(src_geo, tgt_geo,
                                 (view_layout[src].x(), view_layout[src].y()),
                                 (view_layout[tgt].x(), view_layout[tgt].y()))
    })
print(len(edges))

21093


In [102]:
bbox = tlp.computeBoundingBox(final_layout)

In [103]:
# Use the mapping function from shapely to serialize the geometry objects
for n in leaf_nodes:
    n['geometry'] = mapping(n['geometry'])
for e in edges:
    e['geometry'] = mapping(e['geometry'])
for _, n in metanodes.items():
    n['geometry'] = mapping(n['geometry'])

json_data = {
    'leaf_nodes': leaf_nodes,
    'edges': edges,
    'height': 2 if len(leaf_nodes) > 0 else 1,
    'root': root,
    'metanodes': metanodes,
    'bounding_box': [[bbox[0].x(), bbox[0].y()], [bbox[1].x(), bbox[1].y()]]
}

In [104]:
json.dump(json_data, open(myjson_filepath, 'w'), indent=2)
print('Converted to ', myjson_filepath, ' #nodes:', len(leaf_nodes), ' #edges: ', len(edges), 
      '#metanodes:', len(metanodes),
      ' height: ', json_data['height'])

Converted to  ../../../data/real-world-compiled/partition/partition-bcsstk33-grouseflocks-open-8.json  #nodes: 466  #edges:  21093 #metanodes: 200  height:  2
